In [ ]:
from warnings import filterwarnings

filterwarnings("ignore")

In [ ]:
!pip install catboost
!pip install gplearn
!pip install tbats
!pip install pmdarima
!pip install etna

In [ ]:
from datetime import datetime
import typing as tp

from IPython.display import display
from pylab import rcParams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

import xgboost
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import statsmodels
import statsmodels.api as sm
from statsmodels.stats.proportion import proportion_confint
from statsmodels.tsa.seasonal import seasonal_decompose, STL

from gplearn.genetic import SymbolicRegressor
from tbats import TBATS, BATS
from pmdarima import auto_arima

from etna.metrics import SMAPE
from etna.pipeline import AutoRegressivePipeline


rcParams["figure.figsize"] = 15, 7

sns.set(palette="Set2", font_scale=1.3)

# Сведение задачи прогнозирования временного ряда к регрессии

In [ ]:
df1 = pd.read_csv('dataset_v0.csv', parse_dates=['SaleDateTime'], dayfirst=True)
df2 = pd.read_csv('dataset.csv', parse_dates=['SaleDateTime'], dayfirst=True)
df2.drop(columns=['Unnamed: 0', 'CheckDateTime'], inplace=True)
df3 = pd.read_csv('result_615246.csv', parse_dates=['SaleDateTime'], dayfirst=True)
df3.drop(columns=['Unnamed: 0', 'CheckDateTime'], inplace=True)

df = pd.concat([df1, df2, df3], ignore_index=True)
df.shape

In [ ]:
df.drop(columns=['SaleId', 'DeviceId', 'ShopId', 'CompanyId', 'Sum', 'ServerDateTime', 'IsFiscal'], 
        inplace=True)

df.head(3)

In [ ]:
goods = pd.read_csv('goods.csv')
goods.drop(columns=['CompanyId', 'UnitType', 'TaxType', 'Barcode', 'Price', 'ProductName',
                    'AccuracyType', 'ShortCodes', 'ProductDescription'], inplace=True)

In [ ]:
df = df.merge(goods, how='left', on='ProductId')
df.sample(3)

In [ ]:
df['date'] = df['SaleDateTime'].dt.normalize()
df.drop(columns=['SaleDateTime', 'ProductId', 'PositionId'], axis=1, inplace=True)

In [ ]:
df.groupby('date').sum()['Quantity']

In [ ]:
df['GroupId'].value_counts()

In [ ]:
res = STL(df.groupby('date').sum()['Quantity'], period=7).fit()
res.plot();

## Все теги сразу анализировать не получится: выбираем конкретный и делаем всё для него

In [ ]:
TAG = 213

In [ ]:
df.sort_values('date', inplace=True)
data = df[df['GroupId'] == TAG].drop('GroupId', axis=1)
data = data.groupby('date').sum()


ts = data['Quantity']

### 1.2 Данные
Разделим данные на трейн и тест для обучения и тестирования результатов соотвественно.

In [ ]:
test_size = 14
data_train = ts.iloc[:-test_size]
data_test = ts.iloc[-test_size:]

Визуализируем полученные данные. Визуализировать данные нужно с самого начала. Это помогает провалидировать данные и выделить некоторые закономерности.

In [ ]:
plt.figure(figsize=(15, 5))
plt.title(f'Продажи товара тега {TAG}')
plt.plot(data_train, label="train")
plt.plot(data_test, label="test")
plt.legend();

In [ ]:
result = STL(data_train, period=7).fit()
result.plot();

### Метрики
Прежде чем ее решать, зададим метрики, по которым мы будем определять, какая из моделей лучше: RMSE, MAE, SMAPE.

In [ ]:
compare_table = None

In [ ]:
def calculate_smape(actual, predicted) -> float:
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )


def add_results_in_comparison_table(
    method: str, y_true, y_forecast
) -> pd.DataFrame:
    """
    Добавляет новую строчку в таблицу compare_table
    с результатами текущей модели.

    Если ранее модель была уже добавлена в таблицу,
    то старая строчка перезапишется на новую.
    """

    # Обращаемся к глобальной переменной
    global compare_table

    # Считаем метрики
    result_row = {
        "method": method,
        "RMSE": mean_squared_error(y_true=y_true, y_pred=y_forecast, squared=False),
        "MAE": mean_absolute_error(y_true=y_true, y_pred=y_forecast),
        "SMAPE": calculate_smape(y_true, y_forecast)

    }

    # Записываем результат в таблицу
    if compare_table is None:
        compare_table = pd.DataFrame([result_row])
    else:
        if method in list(compare_table["method"]):
            compare_table = compare_table[compare_table["method"] != method]

        compare_table = pd.concat([compare_table, pd.DataFrame([result_row])])
        compare_table.index = np.arange(len(compare_table))
    return compare_table

In [ ]:
def plot_results(y_to_train, y_to_test, y_forecast, plot_conf_int=False,
                 left_bound=None, right_bound=None):
    """
        Функция для визуализации временного ряда и предсказания.
    """

    plt.figure(figsize=(15, 5))
    plt.title(f'Временной ряд продаж тега {TAG}', fontsize=15)
    plt.plot(y_to_train, label='train')
    plt.plot(y_to_test, label='test')
    plt.plot(y_to_test.index, y_forecast, label='prediction')
    if plot_conf_int:
        plt.fill_between(y_to_test.index, 
                         left_bound, right_bound, 
                         alpha=0.3, color='grey',
                         label='conf.int')
    plt.legend()
    plt.show()

### Работа с признаками 1
 ____
 

Для начала преобразуем дату, выделив из даты день, месяц, год и т.д. Для этого будем использовать функцию ниже.

In [ ]:
def create_date_features(date):
    """Создает фичи из даты"""

    row = {}
    row["dayofweek"] = date.dayofweek
    row["quarter"] = date.quarter
    row["month"] = date.month
    row["year"] = date.year
    row["dayofyear"] = date.dayofyear
    row["dayofmonth"] = date.day
    row["weekofyear"] = date.weekofyear
    return row

In [ ]:
def create_only_date_train_features(y_series):
    """
    Создает обучающий датасет из признаков, полученных из дат для y_series
    """
    time_features = pd.DataFrame(
        [create_date_features(date) for date in y_series.index]
    )
    return time_features, y_series

In [ ]:
X_train, y_train = create_only_date_train_features(data_train)
display(X_train.head())
display(y_train.head())

In [ ]:
def devide_df_by_day(dataframe: pd.DataFrame, time: int = 16) -> tuple[list[pd.Series], list[pd.Series]]:
    """
    Бьёт датасет на train и test части по времени суток
    """
    before_time = []
    after_time = []
    for date in dataframe['date'].unique():
        day_bills = dataframe[dataframe['date'] == date]
        if day_bills.shape[0] == 0:
            continue
        before = day_bills[day_bills['hour'] <= time].loc[:, ['Quantity', 'Price', 'CostPrice']].sum()
        for time_feature_name in time_features:
            before[time_feature_name] = day_bills.iloc[0].loc[time_feature_name]
            
        before_time.append(before)
        
        after = day_bills[day_bills['hour'] > time].loc[:, ['Quantity', 'Price', 'CostPrice']].sum()
        after['date'] = day_bills.iloc[0].loc['date']
        after_time.append(after)
    
    return before_time, after_time

In [ ]:
def hybrid_prediction(
    model, test_dates, y_to_train, features_creation_function
):
    """
    Функция для предсказания для дат, указанных в test_dates.
    """
    predictions = []
    previous_y = list(y_to_train)

    for date in test_dates:
        row = features_creation_function(date, previous_y)
        curr_test = pd.DataFrame([row])
        curr_prediction = round(model.predict(curr_test)[0])
        previous_y.append(curr_prediction)
        predictions.append(curr_prediction)
    return np.array(predictions)

Попробуем метод `RandomForest`. Обучим модель.

In [ ]:
%%time
random_forest = RandomForestRegressor(n_estimators=300, random_state=42)
random_forest.fit(X_train, y_train)

Получим предсказания.

In [ ]:
random_forest_predictions = hybrid_prediction(
    random_forest,
    data_test.index,
    data_train,
    lambda date, previous_y: create_date_features(date),
)

Отобразим результаты.

In [ ]:
plot_results(data_train, data_test, random_forest_predictions)

In [ ]:
add_results_in_comparison_table(
    "RandomForest", data_test.values, random_forest_predictions
)

Попробуем теперь `CatBoost`.

In [ ]:
%%time
catboost_ = CatBoostRegressor()
catboost_.fit(X_train, y_train, verbose=False)

In [ ]:
catboost_predictions = hybrid_prediction(
    catboost_,
    data_test.index,
    data_train,
    lambda date, previous_y: create_date_features(date),
)

In [ ]:
plot_results(data_train, data_test, catboost_predictions)

In [ ]:
add_results_in_comparison_table("CatBoost", data_test.values, catboost_predictions)

### Работа с признаками 2
 На этот раз добавим сдвиги по времени. Таким образом модель сможет использовать информацию из прошлого, для составления прогноза на будущее.

In [ ]:
def create_date_and_shifted_train_features(
    y_series, shifts=5, week_seasonal_shifts=2
):
    """
    Создает обучающий датасет из признаков, полученных из дат
    и значений ряда ранее.
    При этом используются значения ряда со сдвигами
    на неделю и год назад.
    """

    curr_df, y = create_only_date_train_features(y_series)
    curr_df.index = y_series.index

    # применяем сдвиг по дням
    for shift in range(1, shifts + 1):
        curr_df[f"shift_{shift}"] = y_series.shift(shift, axis=0)

    # применяем сдвиг по неделям
    for shift in range(1, week_seasonal_shifts + 1):
        curr_df[f"week_seasonal_shift_{shift}"] = y_series.shift(
            shift * 7, axis=0
        )

    y = y_series

    # удалим первые строчки с nan
    drop_indices = curr_df.index[curr_df.isna().sum(axis=1) > 0]
    curr_df = curr_df.drop(index=drop_indices)
    y = y.drop(index=drop_indices)
    return curr_df, y

In [ ]:
def date_and_shift_features_generator_for_test(date, previous_y):
    """Функция создания признаков из дат и сдвигов ряда для тестовых дат"""

    row = create_date_features(date)
    for shift in range(1, SHIFT + 1):
        row[f"shift_{shift}"] = previous_y[-1 * shift]
    for shift in range(1, WEEK_SHIFT + 1):
        row[f"week_seasonal_shift_{shift}"] = previous_y[-1 * shift * 7]
    return row

Зададим сами сдвиги.

In [ ]:
SHIFT = 5  # дневной сдвиг
WEEK_SHIFT = 2  # недельный сдвиг

Получим новые признаки.

In [ ]:
X_train, y_train = create_date_and_shifted_train_features(
    data_train,
    shifts=SHIFT,
    week_seasonal_shifts=WEEK_SHIFT,
)

In [ ]:
X_train.head(3)

In [ ]:
%%time
shifted_features_random_forest = RandomForestRegressor(
    n_estimators=300, random_state=42
)
shifted_features_random_forest.fit(X_train, y_train)

In [ ]:
shifted_features_random_forest_predictions = hybrid_prediction(
    shifted_features_random_forest,
    data_test.index,
    data_train,
    date_and_shift_features_generator_for_test,
)

In [ ]:
plot_results(
    data_train,
    data_test,
    shifted_features_random_forest_predictions,
)

Тяжело сказать, улучшился результат или нет.

In [ ]:
add_results_in_comparison_table(
    "RandomForest + shift features",
    data_test.values,
    shifted_features_random_forest_predictions,
)

In [ ]:
%%time
shifted_features_catboost = CatBoostRegressor()
shifted_features_catboost.fit(X_train, y_train, verbose=False)

In [ ]:
shifted_features_ctb_predictions = hybrid_prediction(
    shifted_features_catboost,
    data_test.index,
    data_train,
    date_and_shift_features_generator_for_test,
)

In [ ]:
plot_results(
    data_train,
    data_test,
    shifted_features_ctb_predictions,
)

In [ ]:
add_results_in_comparison_table(
    "CatBoost + shift features", data_test.values, shifted_features_ctb_predictions
)

### Работа с признаками 3
Еще немного поэксперементируем с признаками. На этот раз добавим признаки скользящего среднего, максимума и минимума.

In [ ]:
def create_date_shifted_and_rolling_train_features(
    y_series, shifts=5, week_seasonal_shifts=2
):
    """
    Создает обучающий датасет из признаков, полученных из дат
    и значений ряда ранее.
    Используются занчения ряда со сдвигами на неделю и год назад.
    Также добавлены признаки скользящего среднего, минимума и максимума.

    """

    curr_df, y = create_date_and_shifted_train_features(
        y_series,
        shifts=shifts,
        week_seasonal_shifts=week_seasonal_shifts
    )

    curr_df["rolling_mean"] = (
        y_series.rolling(shifts, min_periods=1).mean().shift(1, axis=0)
    )
    curr_df["rolling_max"] = (
        y_series.rolling(shifts, min_periods=1).max().shift(1, axis=0)
    )
    curr_df["rolling_min"] = (
        y_series.rolling(shifts, min_periods=1).min().shift(1, axis=0)
    )

    drop_indices = curr_df.index[curr_df.isna().sum(axis=1) > 0]
    curr_df = curr_df.drop(index=drop_indices)
    y = y.drop(index=drop_indices)
    return curr_df, y

In [ ]:
def date_shifted_and_rolling_features_generator_for_test(date, previous_y):
    """Функция создания признаков из дат исдвигов ряда для тестовых дат"""

    row = date_and_shift_features_generator_for_test(date, previous_y)
    row["rolling_mean"] = np.mean(previous_y[-SHIFT:])
    row["rolling_max"] = np.max(previous_y[-SHIFT:])
    row["rolling_min"] = np.min(previous_y[-SHIFT:])
    return row

In [ ]:
X_train, y_train = create_date_shifted_and_rolling_train_features(
    data_train,
    shifts=SHIFT,
    week_seasonal_shifts=WEEK_SHIFT,
)

In [ ]:
advanced_features_random_forest = RandomForestRegressor(
    n_estimators=300, random_state=42
)
advanced_features_random_forest.fit(X_train, y_train)

In [ ]:
advanced_features_random_forest_predictions = hybrid_prediction(
    advanced_features_random_forest,
    data_test.index,
    data_train,
    date_shifted_and_rolling_features_generator_for_test,
)

In [ ]:
plot_results(
    data_train,
    data_test,
    advanced_features_random_forest_predictions,
)

In [ ]:
add_results_in_comparison_table(
    "RandomForest, advanced features",
    data_test.values,
    advanced_features_random_forest_predictions,
)

Применим теперь `CatBoost` к датасету с новыми признаками.

In [ ]:
advanced_features_catboost = CatBoostRegressor()
advanced_features_catboost.fit(X_train, y_train, verbose=False)

In [ ]:
advanced_features_ctb_predictions = hybrid_prediction(
    advanced_features_catboost,
    data_test.index,
    data_train,
    date_shifted_and_rolling_features_generator_for_test,
)

In [ ]:
plot_results(
    data_train,
    data_test,
    advanced_features_ctb_predictions,
)

In [ ]:
add_results_in_comparison_table(
    "CatBoost, advanced features", data_test.values, advanced_features_ctb_predictions
)

In [ ]:
tbats_estimator = TBATS()
model = tbats_estimator.fit(data_train)

In [ ]:
print(model.summary())

In [ ]:
tbats_forecast, confidence_info = model.forecast(steps=14, confidence_level=0.95)

In [ ]:
plot_results(data_train, data_test, np.round(tbats_forecast), plot_conf_int=True,
             left_bound=confidence_info['lower_bound'],
             right_bound=confidence_info['upper_bound'])

In [ ]:
add_results_in_comparison_table('TBATS model', data_test.values, np.round(tbats_forecast))

In [ ]:
tbats_estimator = TBATS()
tbats_estimator.fit(data_train[-69:])


In [ ]:
tbats_forecast, confidence_info = model.forecast(steps=14, confidence_level=0.95)

In [ ]:
plot_results(data_train[-55:], data_test, tbats_forecast, plot_conf_int=True,
             left_bound=confidence_info['lower_bound'],
             right_bound=confidence_info['upper_bound'])

In [ ]:
add_results_in_comparison_table('TBATS model after november', data_test.values, np.round(tbats_forecast))

In [ ]:
from sklearn.svm import SVC, SVR

svc_model = SVC()
svc_model.fit(X_train, y_train)

advanced_features_svc_predictions = hybrid_prediction(
    svc_model,
    data_test.index,
    data_train,
    date_shifted_and_rolling_features_generator_for_test,
)



In [ ]:
plot_results(data_train, data_test, advanced_features_svc_predictions, plot_conf_int=False)

In [ ]:
add_results_in_comparison_table('SVC', data_test.values, np.round(advanced_features_svc_predictions))

In [ ]:
from sklearn.linear_model import PoissonRegressor

pois_model = PoissonRegressor()
pois_model.fit(X_train, y_train)

advanced_features_pois_predictions = hybrid_prediction(
    pois_model,
    data_test.index,
    data_train,
    date_shifted_and_rolling_features_generator_for_test,
)



In [ ]:
plot_results(data_train, data_test, advanced_features_pois_predictions)

In [ ]:
add_results_in_comparison_table('Pois base', data_test.values, np.round(advanced_features_pois_predictions))

In [ ]:
pois_model = PoissonRegressor(solver='newton-cholesky')
pois_model.fit(X_train, y_train)

advanced_features_pois_predictions = hybrid_prediction(
    pois_model,
    data_test.index,
    data_train,
    date_shifted_and_rolling_features_generator_for_test,
)


In [ ]:
plot_results(data_train, data_test, advanced_features_pois_predictions)

In [ ]:
add_results_in_comparison_table('Pois NH', data_test.values, np.round(advanced_features_pois_predictions))

In [ ]:
def count_losses(max_capacity, strategy_predicted):
     """
     Считает остатки при заданной стратегии с заданной максимальной вместимостью
     """
    return (max_capacity - strategy_predicted).sum()

def print_results_for_all_strategies(prediction):
    """
    Считает и выводит результаты по всем стратегиям
    """
    max_cap = int(max(data_train[-14:].max(), data_test.max()))
    print(max_cap)

    raw_results = count_losses(max_cap, prediction)
    print('Losses for raw predictions:', raw_results)

    min_for_all_consts, best_const = 10000000, 0
    for const in range(1, max_cap):
        res = count_losses(max_cap, prediction + const)
        if res < min_for_all_consts and res > 0:
            min_for_all_consts = res
            best_const = const
    print(f'Losses for predictions with const = {best_const}: {min_for_all_consts}')

    noise = prediction.std()
    random_noises = np.round(np.random.uniform(-noise, noise, prediction.shape[0]))
    print('noise:', noise)
    random_res = count_losses(max_cap, prediction + random_noises)
    print('Losses for predictions with noise:', random_res)

In [ ]:
count_losses(int(max(data_train[-14:].max(), data_test.max())), data_test)

In [ ]:
print_results_for_all_strategies(shifted_features_ctb_predictions)